In [1]:
import dspy


In [2]:
lm = dspy.LM('azure/gpt-4.1', api_key='3c744295edda4f13bf0ef198ddb4d24c', api_base='https://ist-apim-aoai.azure-api.net/load-balancing/gpt-4.1')


In [3]:
dspy.configure(lm=lm)

In [4]:
math = dspy.ChainOfThought("question -> answer: float")
math(question="Two dice are tossed. What is the probability that the sum equals two?")

Prediction(
    reasoning='When two dice are tossed, each die has 6 faces, so there are 6 × 6 = 36 possible outcomes. The sum equals two only if both dice show a 1, i.e., (1,1). This is only 1 outcome out of 36 possible outcomes.\n\nProbability = Number of favorable outcomes / Total number of outcomes = 1 / 36 = 0.027777...',
    answer=0.027777777777777776
)

In [5]:
lm.history

[{'prompt': None,
  'messages': [{'role': 'system',
    'content': 'Your input fields are:\n1. `question` (str):\nYour output fields are:\n1. `reasoning` (str): \n2. `answer` (float):\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## question ## ]]\n{question}\n\n[[ ## reasoning ## ]]\n{reasoning}\n\n[[ ## answer ## ]]\n{answer}        # note: the value you produce must be a single float value\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        Given the fields `question`, produce the fields `answer`.'},
   {'role': 'user',
    'content': '[[ ## question ## ]]\nTwo dice are tossed. What is the probability that the sum equals two?\n\nRespond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## answer ## ]]` (must be formatted as a valid Python float), and then ending with the marker for `[[ ## completed ## ]]`.'}],
  'kwargs': {},
  'response': ModelRe

In [6]:
from typing import Literal

class Emotion(dspy.Signature):
    """Classify emotion."""
    sentence: str = dspy.InputField()
    sentiment: Literal['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'] = dspy.OutputField()

sentence = "i started feeling a little vulnerable when the giant spotlight started blinding me"
classify = dspy.Predict(Emotion)
classify(sentence=sentence)

Prediction(
    sentiment='fear'
)

In [13]:
import dspy

# 1. 连接 ColBERTv2 检索服务（Wikipedia DEMO）
search = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')

# 2. 检索问题
query = "Who designed the Eiffel Tower?"
results = search(query, k=3)

# 3. 打印检索结果
for i, r in enumerate(results):
    print(f"[{i+1}] {r['text']}")

[1] Stephen Sauvestre | Charles Léon Stephen Sauvestre (26 December 1847 - 18 June 1919) was a French architect. He is notable for being one of the architects contributing to the design of the world-famous Eiffel Tower, built for the 1889 Universal Exposition in Paris, France.
[2] Émile Nouguier | Émile Nouguier (17 February 1840 – 23 November 1897) was a French civil engineer and architect. He is famous for co-designing the Eiffel Tower, built 1887–1889 for the 1889 Universal Exposition in Paris, France, the Garabit viaduct, the highest in the world at the time, near Ruynes-en-Margeride, Cantal, France, and the Faidherbe Bridge over the Sénégal River in Senegal.
[3] Eiffel Tower | The Eiffel Tower ( ; French: "tour Eiffel" , ] ) is a wrought iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower.


In [11]:
import os
from mem0 import Memory
os.environ["OPENAI_API_KEY"] = "3c744295edda4f13bf0ef198ddb4d24c"  # embedding用

# 1. 设置环境变量（可选，或直接写在config中）
os.environ["LLM_AZURE_OPENAI_API_KEY"] = "3c744295edda4f13bf0ef198ddb4d24c"
os.environ["LLM_AZURE_DEPLOYMENT"] = "gpt-4.1"  # 你的部署名称
os.environ["LLM_AZURE_ENDPOINT"] = "https://ist-apim-aoai.azure-api.net/load-balancing/gpt-4.1"  # 你的endpoint路径
os.environ["LLM_AZURE_API_VERSION"] = "2024-10-21"  # 你的API版本

# 2. Mem0配置
config = {
    "llm": {
        "provider": "azure_openai",
        "config": {
            "model": os.getenv("LLM_AZURE_DEPLOYMENT"),  # "gpt-4.1"
            # reasoning模型可能不支持temperature、max_tokens，如报错请删除
            "temperature": 0.1,
            "max_tokens": 2000,
            "azure_kwargs": {
                "azure_deployment": os.getenv("LLM_AZURE_DEPLOYMENT"),
                "api_version": os.getenv("LLM_AZURE_API_VERSION"),
                "azure_endpoint": os.getenv("LLM_AZURE_ENDPOINT"),
                "api_key": os.getenv("LLM_AZURE_OPENAI_API_KEY"),
                "default_headers": {
                    # 若无自定义header可为空或删掉
                }
            }
        }
    }
}

# 3. 实例化并使用Memory
m = Memory.from_config(config)

messages = [
    {"role": "user", "content": "I'm planning to watch a movie tonight. Any recommendations?"},
    {"role": "assistant", "content": "How about thriller movies? They can be quite engaging."},
    {"role": "user", "content": "I’m not a big fan of thriller movies but I love sci-fi movies."},
    {"role": "assistant", "content": "Got it! I'll avoid thriller recommendations and suggest sci-fi movies in the future."}
]

m.add(messages, user_id="alice", metadata={"category": "movies"})


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: 3c744295********************d24c. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}